# Unigram Inverted Index

**Imports**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import json
import string
import re
from nltk.stem import PorterStemmer

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load data**

In [5]:
raw_data =json.load(open("/content/drive/MyDrive/IR_Assignments/docs.json", "r"))
file_types =json.load(open("/content/drive/MyDrive/IR_Assignments/special_docs.json", "r"))

**Preprocessing**

In [6]:
def preprocess(input):
    input=input.replace('\a',' ')
    input=input.replace('\b',' ')
    input=input.replace('\f',' ')
    input=input.replace('\n',' ')    
    input=input.replace('\r',' ')
    input=input.replace('\t',' ')
    input=input.replace('\v',' ')
    # removing special characters
    # output = re.sub(r'[^\x20-\x7e]','',input)
    # print(output)
    # convert to lower case
    output = input.lower()
    # remove punctuations
    punctuations=string.punctuation.replace("'",'')
    output = "".join([char if char not in punctuations else ' ' for char in output])
    output = output.replace("'",'')
    # output = "".join([char for char in output if char not in "'"])
    # output = " ".join([char for char in output if char not in string.punctuation.replace("'", "")])
    # tokenize
    output = nltk.word_tokenize(output)
    # removing words with special characters
    output = [word for word in output if re.sub(r'[^\x20-\x7e]','',word) == word]
    # remove stopwords and numeric tokens
    output = [word.strip() for word in output if word not in nltk.corpus.stopwords.words('english') and not word.isnumeric()]
    # stemming
    # output = [PorterStemmer().stem(word) for word in output]
    return output

In [7]:
# preprocessing
for doc in raw_data:
    raw_data[doc] = preprocess(raw_data[doc])

**Creating doc to doc-id mapping**

In [8]:
# doc to doc-id mapping
def map_docs(raw_data):
    doc_ids = {}
    id = 1
    for doc in raw_data:
        doc_ids[doc] = id
        id += 1
    return doc_ids

doc_ids = map_docs(raw_data)

**Creating unigram inverted index**

In [9]:
# creating unigram inverted index
def create_index(doc_ids):
    index = {}
    for doc in doc_ids:
        for token in raw_data[doc]:
            # if token exists in index, add doc id
            if token in index.keys():
                index[token][1].add(doc_ids[doc])
                index[token][0] = len(index[token][1])
            # if token does not exist in index, add to index
            else:
                index[token] = [1, {doc_ids[doc]}]
    return index

index = create_index(doc_ids)

**Operations for queries**

In [10]:
# merging
# OR queries
def merge_or(list1, list2):
    ctr = 0
    result = set()
    docids1 = list(list1[1])
    docids2 = list(list2[1])
    docids1.sort()
    docids2.sort()
    ptr1 = 0
    ptr2 = 0
    len1 = list1[0]
    len2 = list2[0]
    while ptr1 < len1 and ptr2 < len2:
        ctr += 1
        if docids1[ptr1] == docids2[ptr2]:
            result.add(docids1[ptr1])
            ptr1 += 1
            ptr2 += 1
        else:
            # ctr += 1
            if docids1[ptr1] < docids2[ptr2]:
                result.add(docids1[ptr1])
                ptr1 += 1
            else:
                result.add(docids2[ptr2])
                ptr2 += 1

    while ptr1 < len1:
        result.add(docids1[ptr1])
        ptr1 += 1
    while ptr2 < len2:
        result.add(docids2[ptr2])
        ptr2 += 1

    # for val in docids1:
    #     result.add(val)
    # for val in docids2:
    #     result.add(val)
    result_list = []
    result_list.append(len(result))
    result_list.append(result)
    return result_list,ctr

# AND queries
def merge_and(list1, list2):
    ctr = 0
    result = set()
    ptr1 = 0
    ptr2 = 0
    len1 = list1[0]
    len2 = list2[0]
    docids1 = list(list1[1])
    docids2 = list(list2[1])
    docids1.sort()
    docids2.sort()
    while ptr1 < len1 and ptr2 < len2:
        ctr += 1
        if docids1[ptr1] == docids2[ptr2]:
            result.add(docids1[ptr1])
            ptr1 += 1
            ptr2 += 1
        else:
            # ctr += 1
            if docids1[ptr1] < docids2[ptr2]:
                ptr1 += 1
            else:
                ptr2 += 1
    result_list = []
    result_list.append(len(result))
    result_list.append(result)
    return result_list,ctr

# AND NOT queries
def merge_andnot(list1, list2):
    ctr = 0
    result = set()
    ptr1 = 0
    ptr2 = 0
    len1 = list1[0]
    len2 = list2[0]
    docids1 = list(list1[1])
    docids2 = list(list2[1])
    docids1.sort()
    docids2.sort()
    while ptr1 < len1 and ptr2 < len2:
        ctr += 1
        if docids1[ptr1] < docids2[ptr2]:
            result.add(docids1[ptr1])
            ptr1 += 1
        elif docids1[ptr1] > docids2[ptr2]:
            ptr2 += 1
            # ctr += 1
        else:
            ptr1 += 1
            ptr2 += 1
            # ctr += 1
    result_list = []
    result_list.append(len(result))
    result_list.append(result)
    return result_list,ctr  

# OR NOT queries
def merge_ornot(list1, list2):
    ctr = 0
    result = set()
    docids1 = list(list1[1])
    docids2 = list(list2[1])
    docids1.sort()
    docids2.sort()
    ptr1 = 0
    ptr2 = 0
    len1 = list1[0]
    len2 = list2[0]
    while ptr1 < len1 and ptr2 < len2:
        ctr += 1
        if docids1[ptr1] == docids2[ptr2]:
            if ptr2 == 0:
              for i in range(1,docids2[ptr2]+1):
                result.add(i)
            else:
              for i in range(docids2[ptr2-1]+1,docids2[ptr2]+1):
                result.add(i)
            result.add(docids1[ptr1])
            ptr1 += 1
            ptr2 += 1
        else:
            # ctr +=1
            if docids1[ptr1] > docids2[ptr2]:
              if ptr2 == 0:
                for i in range(1,docids2[ptr2]):
                  result.add(i)
              else:
                for i in range(docids2[ptr2-1]+1,docids2[ptr2]):
                  result.add(i)
              ptr2 += 1
            else:
                ptr1 += 1
    while ptr2 < len2:
      for i in range(docids2[ptr2-1]+1,docids2[ptr2]):
        result.add(i)     
      ptr2 += 1
    for i in range(docids2[ptr2-1]+1,len(doc_ids)+1):
      result.add(i)   
    # for val in docids1:
    #     result.add(val)
    # all_docs = doc_ids.values()
    # for val in all_docs:
    #     if val not in docids2:
    #         result.add(val)
    result_list = []
    result_list.append(len(result))
    result_list.append(result)
    return result_list,ctr

**Processing the input query and displaying results**

In [11]:
# process the input query and operation sequence
def process(input, operations):
    comparisons = 0
    check_order = 0
    if operations.count("OR") == len(operations) or operations.count("AND") == len(operations):
        check_order = 1
    terms = preprocess(input)
    lists = []
    for term in terms:
        if term in index:
            lists.append(index[term])
        else:
            lists.append([0, {}])

    list1 = lists[0]
    for i in range(0, len(operations), 1):
        scount = 0
        if check_order == 1:
            if i > 0:
                lists.append(list1)
            lists.sort()
            # print(lists)
            list1 = lists.pop(0)
            list2 = lists.pop(0)
        else:
            list2 = lists[i+1]

        operation = operations[i]
        if operation == "OR":
            list1,scount = merge_or(list1, list2)
        elif operation == "AND":
            list1,scount = merge_and(list1, list2)
        elif operation == "AND NOT":
            list1,scount = merge_andnot(list1, list2)
        elif operation == "OR NOT":
            list1,scount = merge_ornot(list1, list2)
        comparisons+=scount
    return list1, comparisons

In [12]:
# assumption: input operation sequence is a comma+space separated string
# input and output
def run():
    n = int(input("Number of queries: "))
    results = []
    for i in range(n):
        query = input("Input sentence: ")
        op_seq = input("Input operation sequence: ").split(', ')
        results.append(process(query, op_seq))
    return results

results = run()

Number of queries: 2
Input sentence: lion stood thoughtfully for a moment
Input operation sequence: OR, OR, OR
Input sentence: telephone,paved, roads
Input operation sequence: OR NOT, AND NOT


In [13]:
# display results
def display_results(result):
    key_list = list(doc_ids.keys())
    val_list = list(doc_ids.values())
    print("Number of documents matched: ", result[0][0])
    print("Number of Comparisons: ", result[1])
    # print("List of documents matched: ")
    # for doc in result[0][1]:
    #     position = val_list.index(doc)
    #     print(key_list[position])

for result in results:
    display_results(result)

Number of documents matched:  182
Number of Comparisons:  298
Number of documents matched:  1103
Number of Comparisons:  1166
